<a href="https://colab.research.google.com/github/StarkRitam02/merchify/blob/main/LibraryManagementChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok faiss-cpu gpt4all groq huggingface-hub langchain langchain-community langchain-core langchain-groq langchain-openai langchain-text-splitters langserve langsmith sentence-transformers tokenizers transformers uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
%%writefile app.py
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.document_loaders.csv_loader import UnstructuredCSVLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq
import numpy as np

# Load your documents
loader = UnstructuredCSVLoader(
    file_path="/content/sample_data/final_library_books.csv", mode="elements"
)
docs = loader.load()
text_documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
documents = text_splitter.split_documents(text_documents)

huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}
embeddings = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
)

vectorstore = FAISS.from_documents(documents, huggingface_embeddings)
db = FAISS.from_documents(documents, GPT4AllEmbeddings(model_name='all-MiniLM-L6-v2.gguf2.f16.gguf'))

prompt_template = """
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

groq_api_key = "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"
llm = ChatGroq(groq_api_key=groq_api_key, model_name="gemma-7b-it")

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":16})

retrievalQA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

# Streamlit app
st.title("Library Management System with AI Integration")

query = st.text_input("Enter your query:")
if query:
    result = retrievalQA.invoke({"query":query})
    st.write(result['result'])
    # st.write("Relevant Documents:")
    # for doc in result['source_documents']:
        # st.write(doc.page_content)


Writing app.py


In [ ]:
# from pyngrok import ngrok

# # Run Streamlit in the background
# !streamlit run app.py &

# # Expose the Streamlit server via ngrok
# public_url = ngrok.connect(port='8501')
# print(f'Streamlit app running at: {public_url}')


ModuleNotFoundError: No module named 'pyngrok'

In [ ]:
# # Install the necessary packages
# !pip install streamlit pyngrok faiss-cpu gpt4all groq huggingface-hub langchain langchain-community langchain-core langchain-groq langchain-openai langchain-text-splitters langserve langsmith sentence-transformers tokenizers transformers uvicorn

# # Write your Streamlit app to a file
# %%writefile app.py
# import streamlit as st
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import GPT4AllEmbeddings
# from langchain_community.document_loaders.csv_loader import UnstructuredCSVLoader
# from langchain.prompts import PromptTemplate
# from langchain.chains import RetrievalQA
# from langchain_groq import ChatGroq
# import numpy as np

# # Load your documents
# loader = UnstructuredCSVLoader(
#     file_path="/content/sample_data/library_management_system_large.csv", mode="elements"
# )
# docs = loader.load()
# text_documents = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
# documents = text_splitter.split_documents(text_documents)

# huggingface_embeddings = HuggingFaceBgeEmbeddings(
#     model_name="sentence-transformers/all-MiniLM-L6-v2",
#     model_kwargs={'device':'cpu'},
#     encode_kwargs={'normalize_embeddings':True}
# )

# model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
# gpt4all_kwargs = {'allow_download': 'True'}
# embeddings = GPT4AllEmbeddings(
#     model_name=model_name,
#     gpt4all_kwargs=gpt4all_kwargs
# )

# vectorstore = FAISS.from_documents(documents, huggingface_embeddings)
# db = FAISS.from_documents(documents, GPT4AllEmbeddings(model_name='all-MiniLM-L6-v2.gguf2.f16.gguf'))

# prompt_template = """
# Use the following piece of context to answer the question asked.
# Please try to provide the answer only based on the context

# {context}
# Question:{question}

# """

# prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# groq_api_key = "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"
# llm = ChatGroq(groq_api_key=groq_api_key, model_name="gemma-7b-it")

# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":16})

# retrievalQA = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=retriever,
#     return_source_documents=True,
#     chain_type_kwargs={"prompt":prompt}
# )

# # Streamlit app
# st.title("Library Management System with AI Integration")

# query = st.text_input("Enter your query:")
# if query:
#     result = retrievalQA.invoke({"query":query})
#     st.write(result['result'])
#     st.write("Relevant Documents:")
#     for doc in result['source_documents']:
#         st.write(doc.page_content)

# # Run the Streamlit app using pyngrok
# from pyngrok import ngrok

# # Run Streamlit in the background
# !streamlit run app.py &

# # Expose the Streamlit server via ngrok
# public_url = ngrok.connect(port='8501')
# print(f'Streamlit app running at: {public_url}')


UsageError: Line magic function `%%writefile` not found.


In [3]:
! pip install streamlit -q

In [5]:
!wget -q -O - ipv4.icanhazip.com

34.124.252.52


In [4]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.1 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=36f39e38698f6aad8e07c192a6e79d78aeac2ff9d1e198183f72fa0ba3e2381a
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e

In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.124.252.52:8501

npx: installed 22 in 3.75s
your url is: https://green-beers-mate.loca.lt
2024-06-29 15:46:23.806642: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 15:46:23.806698: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 15:46:23.909712: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-29 15:46:23.935036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized